## Script de extração de dados do JusBrasil

In [1]:
import os
import pandas as pd
import numpy as np
import camelot
import matplotlib.pyplot as plt
import matplotlib
import requests
import bs4
from bs4 import BeautifulSoup
from pdfminer.high_level import extract_text
import tempfile
import fitz
import re

In [2]:
anexos = pd.read_excel('docs/jusbrasil.xlsx', sheet_name="Anexos")

In [3]:
anexos_copia_min = anexos.copy().loc[:,["processoID", "processoAnexoID", "Download copia", "Tipo de anexo", "Tribunal"]]
sentencas = anexos_copia_min.loc[anexos["Tipo de anexo"] == "SENTENCA", ["processoID", "processoAnexoID", "Download copia", "Tribunal"]]

In [4]:
sentencas

,processoID,processoAnexoID,Download copia,Tribunal
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,TJAM
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,TJAM
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...,TJAM
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...,TJAM
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...,TJAM
...,...,...,...,...
13643,494408691,2315018026,http://jud-anexos.digesto.com.br/8ab01c9d05320...,TRF1
13876,494410810,269258456,http://jud-anexos.digesto.com.br/e55b90bdcd194...,TRF1
13877,494410810,2273754745,http://jud-anexos.digesto.com.br/e3f977a11a4d1...,TRF1
13903,549928831,1874259862,http://jud-anexos.digesto.com.br/4c97bd2fdb061...,TRF1


### Capturando os números dos processos por regex

In [5]:
def captura_num_processos(sentencas):
    copia_sentencas = sentencas.copy()
    sucessos = 0
    processos = []

    for i in range(copia_sentencas.shape[0]):
        link = copia_sentencas.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""
        num_processo = "Não encontrado"

        if 'pdf' in content_type:
            # print(f"[PDF] Extraindo de: {link}")
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with fitz.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
            
        elif 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        match = re.search(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto)
        if match:
            num_processo = match.group(0)
            sucessos += 1

        processos.append(num_processo)
    
    print(f"Números de processos capturados: {sucessos}")

    copia_sentencas["num_processo"] = processos
    return copia_sentencas
    
sentencas_com_num = captura_num_processos(sentencas)
sentencas_com_num

Números de processos capturados: 381


,processoID,processoAnexoID,Download copia,Tribunal,num_processo
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,TJAM,0600357-89.2021.8.04.5600
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,TJAM,0600384-72.2021.8.04.5600
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...,TJAM,0681058-19.2022.8.04.0001
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...,TJAM,0471756-76.2024.8.04.0001
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...,TJAM,0097690-14.2024.8.04.1000
...,...,...,...,...,...
13643,494408691,2315018026,http://jud-anexos.digesto.com.br/8ab01c9d05320...,TRF1,1000098-88.2019.4.01.3604
13876,494410810,269258456,http://jud-anexos.digesto.com.br/e55b90bdcd194...,TRF1,1002190-78.2020.4.01.3903
13877,494410810,2273754745,http://jud-anexos.digesto.com.br/e3f977a11a4d1...,TRF1,1002190-78.2020.4.01.3903
13903,549928831,1874259862,http://jud-anexos.digesto.com.br/4c97bd2fdb061...,TRF1,1000652-26.2020.4.01.3818


In [6]:
sentencas_com_num.loc[sentencas_com_num['num_processo'] != "Não encontrado", :]['num_processo'].unique()

array(['0600357-89.2021.8.04.5600', '0600384-72.2021.8.04.5600',
       '0681058-19.2022.8.04.0001', '0471756-76.2024.8.04.0001',
       '0097690-14.2024.8.04.1000', '0600592-22.2022.8.04.5600',
       '5284867-26.2022.8.09.0011', '5580808-19.2022.8.09.0011',
       '5061164-56.2024.8.09.0051', '5589610-60.2024.8.09.0132',
       '5983628-49.2024.8.09.0051', '5921614-29.2024.8.09.0051',
       '1000009-75.2018.4.01.3903', '1000305-06.2018.4.01.3901',
       '1003105-10.2017.4.01.3200', '5096053-69.2018.8.09.0011',
       '5125255-42.2023.8.09.0100', '5456795-74.2023.8.09.0087',
       '5687936-95.2023.8.09.0130', '5842003-89.2023.8.09.0074',
       '0000156-20.2023.8.02.0076', '0700947-45.2023.8.02.0076',
       '0700160-79.2024.8.02.0076', '0700066-27.2024.8.02.0143',
       '0700580-84.2024.8.02.0076', '0700971-67.2023.8.02.0078',
       '0233366-06.2020.8.06.0001', '0050002-26.2020.8.06.0035',
       '5000132-08.2021.8.08.0011', '5016661-58.2024.8.08.0024',
       '5026374-24.2024.8

### Filtrando processos relevantes, georreferências, áreas afetadas e valores de multa

In [7]:
def captura_melhorada(sentencas):
    copia_sentencas = sentencas.copy()
    processos = 0
    georreferencias = 0
    extensoes = 0
    multas = 0
    processos_lista = []
    georreferencias_lista = []
    extensoes_lista = []
    multas_lista = []
    # palavras_chave_ambientais = [
    #     "impacto ambiental", "crime ambiental", "meio ambiente", "desmatamento",
    #     "poluição", "resíduo tóxico", "reparação ambiental", "contaminação"
    # ]

    for i in range(copia_sentencas.shape[0]):
        link = copia_sentencas.iloc[i, 2]
        response = requests.get(link)
        response.encoding = 'utf-8'
        content_type = response.headers.get('Content-Type', '')
        texto = ""
        process_number = "Não encontrado"
        georref = "Não encontrado"
        extension = "Não encontrado"
        fine = "Não encontrado"

        if 'pdf' in content_type:
            # print(f"[PDF] Extraindo de: {link}")
            # Criando arquivo temporário para ler conteúdo do PDF 
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
                tmp_file.write(response.content)
                tmp_path = tmp_file.name

            # Abrindo arquivo temporário e extraindo texto 
            with fitz.open(tmp_path) as doc:
                for page in doc:
                    texto += page.get_text()
            
        elif 'html' in content_type:
            # print(f"[HTML] Extraindo de: {link}")
            soup = BeautifulSoup(response.content, 'html.parser')
            texto = soup.get_text()

        # Número do processo
        match = re.search(r'\d{7}-\d{2}\.\d{4}\.\d\.\d{2}\.\d{4}', texto)
        if match:
            process_number = match.group(0)
            processos += 1

        # Georreferência
        match = re.search(r'\d{2}[º°]\d{2}[’\']\d{2}[”"]?\s*[NS],?\s*\d{2,3}[º°]\d{2}[’\']\d{2}[”"]?\s*[OLW]', texto)
        if match:
            georref = match.group(0)
            georreferencias += 1

        # Extensão afetada
        match = re.search(r'\d+(?:,\d+)?\s*(?:hectares|km²|m²|ha)', texto, re.IGNORECASE)
        if match:
            extension = match.group(0).strip()
            extensoes += 1

        # Multa (se existir)
        match = re.search(r'R\$ ?\d{1,3}(?:\.\d{3})*,\d{2}', texto)
        if match:
            fine = match.group(0).strip()
            multas += 1

        processos_lista.append(process_number)
        georreferencias_lista.append(georref)
        extensoes_lista.append(extension)
        multas_lista.append(fine)
    
    print(f"Números de processos capturados: {processos}")
    print(f"Números de georreferências capturadas: {georreferencias}")
    print(f"Números de extensões capturadas: {extensoes}")
    print(f"Números de multas capturadas: {multas}")

    copia_sentencas["num_processo"] = processos_lista
    copia_sentencas["georreferência"] = georreferencias_lista
    copia_sentencas["extensão"] = extensoes_lista
    copia_sentencas["multa"] = multas_lista
    return copia_sentencas

sentencas_v2 = captura_melhorada(sentencas)
sentencas_v2

Números de processos capturados: 381
Números de georreferências capturadas: 1
Números de extensões capturadas: 47
Números de multas capturadas: 279


,processoID,processoAnexoID,Download copia,Tribunal,num_processo,georreferência,extensão,multa
1,474474022,2344968081,http://jud-anexos.digesto.com.br/0cacd6d80c499...,TJAM,0600357-89.2021.8.04.5600,Não encontrado,Não encontrado,Não encontrado
7,474474806,2294638461,http://jud-anexos.digesto.com.br/90df1f9ac9917...,TJAM,0600384-72.2021.8.04.5600,07º43’11” S 61º28’11” W,"121,15ha","R$ 1.301.393,30"
33,575718248,1511375012,http://jud-anexos.digesto.com.br/c89c7a26febcc...,TJAM,0681058-19.2022.8.04.0001,Não encontrado,Não encontrado,Não encontrado
87,580649179,1504080402,http://jud-anexos.digesto.com.br/585fdfebfdd6d...,TJAM,0471756-76.2024.8.04.0001,Não encontrado,Não encontrado,"R$8.000,00"
105,654928926,2419043700,http://jud-anexos.digesto.com.br/6be7e9beac86d...,TJAM,0097690-14.2024.8.04.1000,Não encontrado,Não encontrado,"R$ 8.971,31"
...,...,...,...,...,...,...,...,...
13643,494408691,2315018026,http://jud-anexos.digesto.com.br/8ab01c9d05320...,TRF1,1000098-88.2019.4.01.3604,Não encontrado,"422,58 hectares","R$ 10.742,00"
13876,494410810,269258456,http://jud-anexos.digesto.com.br/e55b90bdcd194...,TRF1,1002190-78.2020.4.01.3903,Não encontrado,"71,9 hectares","R$ 1.160.136,00"
13877,494410810,2273754745,http://jud-anexos.digesto.com.br/e3f977a11a4d1...,TRF1,1002190-78.2020.4.01.3903,Não encontrado,"71,9 hectares","R$ 10.000.000,00"
13903,549928831,1874259862,http://jud-anexos.digesto.com.br/4c97bd2fdb061...,TRF1,1000652-26.2020.4.01.3818,Não encontrado,Não encontrado,Não encontrado
